<a href="https://colab.research.google.com/github/MHosseinHashemi/Image_Similarity/blob/main/Image_Simmilarity_TF_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

from tqdm import tqdm
from collections import defaultdict

In [ ]:
(train_data, test_data, validation_data), info = tfds.load("oxford_flowers102", split=['train', 'validation', 'test'], as_supervised=True, with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1020 [00:00<?, ? examples/s]

...

Generating test examples...:   0%|          | 0/6149 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_flowers102/2.1.1.incompleteR72LC5/oxford_flowers102-test.tfrecord*.…

Generating validation examples...:   0%|          | 0/1020 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_flowers102/2.1.1.incompleteR72LC5/oxford_flowers102-validation.tfre…

Dataset oxford_flowers102 downloaded and prepared to /root/tensorflow_datasets/oxford_flowers102/2.1.1. Subsequent calls will reuse this data.


In [ ]:
height = 128
width = 128

def preprocess_images(image, label, height, width):
    # image = tf.image.resize_with_crop_or_pad(image, target_height=height, target_width=width)
    image = tf.image.resize(image, [width, height])
    image = tf.cast(image, tf.float32) / 255.0
    return image, label


In [ ]:
train_ds = train_data.map(lambda image, label: preprocess_images(image, label, height, width))

In [ ]:
test_ds = test_data.map(lambda image, label: preprocess_images(image, label, height, width))

In [ ]:
def data_loader(data):
  x = []
  y = []
  for img, label in tqdm(data.as_numpy_iterator()):
    x.append(img)
    y.append(label)

  return x, y

In [ ]:
x_train, y_train = data_loader(train_ds)

1020it [00:03, 282.57it/s]


In [ ]:
x_test, y_test = data_loader(test_ds)

1020it [00:02, 477.92it/s]


In [ ]:
# Model
MODEL_URL = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_s/feature_vector/2"

model = tf.keras.Sequential([
    tf.keras.layers.RandomFlip(),
    tf.keras.layers.RandomRotation(0.3),
    hub.KerasLayer(MODEL_URL, trainable=True),
    tf.keras.layers.Dropout(0.25),
    # tf.keras.layers.Dense(128, activation=None),
    tf.keras.layers.Dense(102, activation='softmax')
    # tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings
])

model.build([None, height, height, 3])
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip_1 (RandomFlip)  (None, 128, 128, 3)       0         
                                                                 
 random_rotation_1 (RandomRo  (None, 128, 128, 3)      0         
 tation)                                                         
                                                                 
 keras_layer_1 (KerasLayer)  (None, 1280)              20331360  
                                                                 
 dropout_1 (Dropout)         (None, 1280)              0         
                                                                 
 dense_1 (Dense)             (None, 102)               130662    
                                                                 
Total params: 20,462,022
Trainable params: 20,308,150
Non-trainable params: 153,872
____________________________________

In [ ]:
def batch_me(images, labels, batch_size, samples_per_class):
  temp_dict = defaultdict(list) # A Dic of Lists to save img, label pairs as one object
  for img, label in zip(images, labels):
    temp_dict[label].append(img)

  while True:
    batch_x = []
    batch_y = []
    while len(batch_x) < batch_size:
      for category, examples in temp_dict.items():
        # Only feed as large as the "samples per class"
        # If the batch did not had enough space, feed as much as it has
        n_samples = min(samples_per_class, (batch_size - len(batch_x)))
        if n_samples == 0:
          break
        # Pick randomly from simmilar images of the same category
        samples = random.sample(examples, k=n_samples)
        # Add corresponding x, y values to the batch
        batch_x.extend(samples)
        batch_y.extend([category] * len(samples))

    # It should be a continous operation
    yield np.array(batch_x), np.array(batch_y)


In [ ]:
def center_loss(feature_vector, center):
    difference = feature_vector - center
    loss = tf.reduce_mean(tf.reduce_sum(difference**2, axis=1))

    return loss

In [ ]:
epochs = 50
alpha = 0.5
batch_size = 16
n_examples_per_class = 4
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

centers = tf.Variable(initial_value=tf.random.normal((102, 128), mean=0.0, stddev=0.5))

for epoch in tqdm(range(epochs)):
    total_loss = 0.0
    num_batches = 0
    for batch_x, batch_y in batch_me(images=x_train, labels=y_train, batch_size=batch_size, samples_per_class=n_examples_per_class):
        # Capture Gradients
        with tf.GradientTape() as tape:
            # Extract Features per batch
            temps = model(batch_x, training=True)
            # Seprate labels and their features
            predictions = temps[0]
            features = temps[1]
            # create dummy center variable
            batch_centers = tf.Variable(initial_value=tf.random.normal((16, 128), mean=0.0, stddev=0.5), trainable=False)
            # and fill it with centers calculated from each faeture vector
            for index in batch_y:
                batch_centers[index] = np.mean(features[index], axis=0)

            # Center-Loss calculation
            c_loss = center_loss(features, batch_centers)
            # Combine it with CategoricalCrossEntropyLoss
            cls_loss = tf.keras.losses.CategoricalCrossentropy()(batch_y, predictions)
            # Total Loss
            loss = (c_loss * alpha) + cls_loss

        # Calculate Gradients
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        # Update Center Values
        for i, category in enumerate(batch_y):
            centers[category].assign_add(batch_centers[i])

        # Update training loss
        total_loss += loss
        num_batches += 1

    # Calculate training Loss
    training_loss = total_loss / num_batches


    """Validation"""
    val_temps = model(x_test)
    # Extract Validation Features and Labels
    val_predictions = val_temps[0]
    val_features = val_temps[1]

    # Calculate Centers
    val_batch_centers = tf.Variable(initial_value=tf.random.normal((16, 128), mean=0.0, stddev=0.5), trainable=False)
    for i in y_test:
        val_batch_centers[i] = np.mean(val_features[i], axis=0)

    # Calculate Loss values
    val_c_loss = center_loss(val_features, val_batch_centers)
    val_cls_loss = tf.keras.losses.CategoricalCrossentropy()(y_test, val_predictions)
    val_loss = (val_c_loss * alpha) + val_cls_loss

    print(f"Epoch {epoch + 1}/{epochs} - Training Loss: {training_loss:.4f} - Validation Loss: {val_loss.numpy():.4f}")



# **TODO**

*The provided code seems to be an attempt to train a model for image similarity using both center loss and categorical cross-entropy loss on the Oxford 102 flowers dataset. Overall, the code structure and approach are reasonable, but there are a few areas that could be improved or clarified*:


- **Center Update**: The code updates the center values using the centers variable, but it initializes this variable with a random normal distribution. It might be more effective to initialize the centers with zeros, as this provides a better starting point.

- **Center Calculation**: The code calculates the batch centers for each batch using the mean of the feature vectors. However, in this implementation, the same center values are updated for each category within the batch. It's more common to calculate centers per category over the entire training set and then update them as the training progresses.

- **Learning Rate Schedule**: Consider using a learning rate schedule, such as the tf.keras.optimizers.schedules module, to adjust the learning rate during training. This can help with convergence and stability.

- **Batch Size and Center Calculation**: The current code calculates centers based on batch_y, which means it uses the current batch for calculating centers. This might lead to unstable center values. It's better to accumulate the feature vectors for each category over the entire training set and then calculate centers after an epoch.

- **Batch Normalization**: The model uses a pretrained EfficientNet model, which is a good choice. However, consider using batch normalization layers after the hub layer to improve training stability.

- **Variable Names**: Some variable names could be more descriptive. For example, temps and val_temps might be better named as outputs or something similar.

- **Validation Loop**: The validation loop calculates the loss for the entire validation set but doesn't update any values. It's common to calculate metrics such as accuracy as well.

- **Center Loss Term**: The alpha parameter is used to weight the center loss term. You might need to experiment with this value to achieve a good balance between the two loss components.

- **Normalization**: While the preprocessing of images includes scaling them to the range [0, 1], you may also consider standardizing the images (subtracting mean and dividing by standard deviation) to help convergence.

- **Model Checkpoints**: Consider using model checkpoints to save the best model during training based on validation performance.